In [1]:
import pandas as pd
from scipy import stats
from statsmodels.stats.multicomp import pairwise_tukeyhsd
import seaborn as sns

#https://towardsdatascience.com/anova-tukey-test-in-python-b3082b6e6bda
experiment_names = ['owl_exp_pure',
                    'owl_exp_mixed',
                    'strike_exp_pure',
                    'strike_exp_mixed',
                    'energy_gain_pure',
                    'energy_gain_mixed',
                    ]

def query_exp_data(label):
    total_exp_data=[]
    for i in range(1,7):
        file_path_totals = '/home/mremington/Documents/uumarrty_exps/{}/exp{}/Data/totals.csv'.format(label,i)
        temp_exp_total = pd.read_csv(file_path_totals,header = 0, index_col=None)
        temp_exp_total['experiment_name'] = [label for i in range(len(temp_exp_total))]
        total_exp_data.append(temp_exp_total)
    total_exp_data = pd.concat(total_exp_data, ignore_index=False)
    return total_exp_data

def column_labels():
    orgs=['krat','snake']
    column_labels=[]
    column_labels.append('id')
    for org in orgs:
        for i in range(1,7):
            label='{}_exp_{}'.format(org,i)
            column_labels.append(label)
    return column_labels

def append_string(fp,string):
    with open(fp, 'a') as f:
        f.write(string+"\n")
        
def p_value_annot(p_val):
    if p_val <= 0.001:
        annot='***'
    elif p_val <= 0.01:
        annot='**'
    elif p_val <= 0.05:
        annot='*'
    else:
        annot=''
    return annot

In [3]:
experiment_names = ['owl_exp_pure',
                    'owl_exp_mixed',
                    'strike_exp_pure',
                    'strike_exp_mixed',
                    'energy_gain_pure',
                    'energy_gain_mixed',
                    ]
txt_file_path='anova_results_04072022.txt'

with open(txt_file_path, 'w') as f:
    pass

for exp in experiment_names:
    intro = "Anova Results for Experiment: {}".format(exp)
    append_string(fp=txt_file_path,string=intro)
    append_string(fp=txt_file_path,
            string="".join(["_" for i in range(len(intro))]))
    append_string(fp=txt_file_path,string='')
    append_string(fp=txt_file_path,string='')
    data = query_exp_data(label=exp)
    # Anova
    pivot_anova = data.pivot_table(index=["sim_number"], 
                            columns=['data_type','experiment'], 
                            values='mean_bush_pref')
    pivot_anova = pivot_anova.reset_index()
    pivot_anova.columns = column_labels()
    #krats
    append_string(fp=txt_file_path,string='Krat')
    krat_fvalue, krat_pvalue = stats.f_oneway(pivot_anova['krat_exp_1'],
                                              pivot_anova['krat_exp_2'],
                                              pivot_anova['krat_exp_3'],
                                              pivot_anova['krat_exp_4'],
                                              pivot_anova['krat_exp_5'],
                                              pivot_anova['krat_exp_6'],)
    append_string(fp=txt_file_path,string='F Value: {}, P Value {}'.format(krat_fvalue, krat_pvalue))
    append_string(fp=txt_file_path,string='')
    #Snakes
    append_string(fp=txt_file_path,string='Snake')
    snake_fvalue, snake_pvalue = stats.f_oneway(pivot_anova['snake_exp_1'],
                                                pivot_anova['snake_exp_2'],
                                                pivot_anova['snake_exp_3'],
                                                pivot_anova['snake_exp_4'],
                                                pivot_anova['snake_exp_5'],
                                                pivot_anova['snake_exp_6'],)
    append_string(fp=txt_file_path,string='F Value: {}, P Value {}'.format(snake_fvalue, snake_pvalue))
    append_string(fp=txt_file_path,string='')
    #Tukey
    intro = "Tukey Results for Experiment: {}".format(exp)
    append_string(fp=txt_file_path,string=intro)
    pivot_tukey = data.pivot_table(index=["experiment","sim_id"], 
                            columns=['data_type'], 
                            values='mean_bush_pref')
    pivot_tukey = pivot_tukey.reset_index()
    pivot_tukey.columns=['experiment','sim_id','krat','snake']
    m_comp_krat = pairwise_tukeyhsd(endog=pivot_tukey['krat'], groups=pivot_tukey['experiment'], alpha=0.05)
    append_string(fp=txt_file_path,string="Krat Tukey")
    append_string(fp=txt_file_path,string=str(m_comp_krat))
    append_string(fp=txt_file_path,string="")
    append_string(fp=txt_file_path,string="")
    m_comp_snake = pairwise_tukeyhsd(endog=pivot_tukey['snake'], groups=pivot_tukey['experiment'], alpha=0.05)
    append_string(fp=txt_file_path,string="Snake Tukey")
    append_string(fp=txt_file_path,string=str(m_comp_snake))
    append_string(fp=txt_file_path,string="")
    append_string(fp=txt_file_path,string="")

In [3]:
data = query_exp_data(label='owl_exp_pure')
pivot_tukey = data.pivot_table(index=["experiment","sim_id"], 
                        columns=['data_type'], 
                        values='mean_bush_pref')
pivot_tukey = pivot_tukey.reset_index()
pivot_tukey.columns=['experiment','sim_id','krat','snake']
m_comp_krat = pairwise_tukeyhsd(endog=pivot_tukey['krat'], groups=pivot_tukey['experiment'], alpha=0.05)

In [61]:
# df = pd.DataFrame(m_comp_krat.summary())
# df.columns = df.iloc[0]
# df = df.iloc[1:].reset_index(drop=True)
# df.head()
from numpy.linalg import inv
tk_krat = m_comp_krat.summary()
a = tk_krat.data

df = pd.DataFrame(a)
df.columns = df.iloc[0]
df = df.iloc[1:].reset_index(drop=True)
temp=list(df.iloc[:,3])

df['annot'] = [p_value_annot(float(p)) for p in temp]
df.columns=['group1','group2','meandiff','p_adj','lower','upper','reject','annot']
df['group1'] = [ int(i[-1]) for i in df['group1']]
df['group2'] = [ int(i[-1]) for i in df['group2']]
df.head()


,group1,group2,meandiff,p_adj,lower,upper,reject,annot
0,1,2,0.0705,0.001,0.062,0.079,True,***
1,1,3,0.1659,0.001,0.1575,0.1744,True,***
2,1,4,0.4926,0.001,0.4842,0.5011,True,***
3,1,5,0.4987,0.001,0.4902,0.5071,True,***
4,1,6,0.4998,0.001,0.4913,0.5083,True,***


In [66]:
data = df[['group1','group2','p_adj']]
data = data.pivot(index='group1', columns='group2', values='p_adj')
#data = data.reset_index()
data.columns = ['ind']
data.head()
# ,linewidths=.5,vmin=-0.1, vmax=0.1, annot=df['annot']
#hmap = sns.heatmap(data,cmap='seismic')
#hmap.set_ylim(0, 5)
#data.head()


group2,2,3,4,5,6
group1,,,,,
1,0.001,0.001,0.001,0.001,0.001
2,NaN,0.001,0.001,0.001,0.001
3,NaN,NaN,0.001,0.001,0.001
4,NaN,NaN,NaN,0.3163,0.1493
5,NaN,NaN,NaN,NaN,0.9


In [52]:
hmap = sns.heatmap(df.iloc[:,0:3],cmap='seismic',linewidths=.5,vmin=-0.1, vmax=0.1, annot=df['annot'])
hmap.set_ylim(0, 5)
#hmap.set_title('Partial correlation')

TypeError: float() argument must be a string or a number, not 'Cell'